2- Se tiene un RDD registros de ventas de producto con la forma (fecha de venta,
código de producto, precio de venta) y en otro RDD detalle de los productos con
(código de producto, descripción del producto, categoría). Se pide resolver utilizando PySpark:

a) Cuál es el producto más vendido.

In [208]:
import pyspark as ps
import pandas as pd

# create the Spark Session
spark = ps.sql.SparkSession.builder.getOrCreate()
# create the Spark Context
sc = spark.sparkContext

aux = sc.textFile('/home/marcos/PycharmProjects/practicaSpark/Ej2a.csv')
salesRdd = aux.map(lambda x: x.split(","))
salesRdd = salesRdd.map(lambda x: (pd.to_datetime(x[0], errors='coerce', format='%Y-%m-%d')
                                   , pd.to_numeric(x[1], errors='coerce'), pd.to_numeric(x[2], errors='coerce')))

aux = sc.textFile('/home/marcos/PycharmProjects/practicaSpark/Ej2b.csv')
infoRdd = aux.map(lambda x: x.split(","))
infoRdd = infoRdd.map(lambda x: (pd.to_numeric(x[0], errors='coerce'), x[1], x[2]))

In [209]:
salesRdd.cache()
infoRdd.cache()

productSales = salesRdd.map(lambda x: (x[1], 1)).reduceByKey(lambda x,y: x + y).cache()
productSales.reduce(lambda x,y: x if (x[1] > y[1]) else y)[0]

19

b) Cuál es la categoría de productos más vendida.

In [210]:
productSalesAndCat = productSales.join(infoRdd.map(lambda x: (x[0], x[2]))).cache()
productSalesAndCat.map(lambda x: (x[1][1], x[1][0])).reduceByKey(lambda x,y: x + y)\
                .reduce(lambda x,y: x if (x[1] > y[1]) else y)[0]

'Comida'

c) Cuál es el top5 de productos más vendidos generando un RDD con
(código de producto, descripción, cantidad de ventas)

In [211]:
top5products = sc.parallelize(productSales.takeOrdered(5, lambda x: x[1]))
top5products.leftOuterJoin(infoRdd).map(lambda x: (x[0], x[1][0], x[1][1])).collect()

[(17, 46, 'tpffqujl'),
 (3, 41, 'gyijzwegw'),
 (14, 44, 'aegubbljs'),
 (6, 46, 'bqxugkkyub'),
 (15, 46, 'ymjjougja')]

d) Cuál es el producto que registró mayor aumento de precio en el último año,
tomando para este análisis solo los productos que cuenten con al menos 50 ventas en el último año.

In [212]:
salesFrom2019 = salesRdd.filter(lambda x: x[0].year == 2019).map(lambda x: (x[1], x[2]))

filteredProducts = salesFrom2019.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x + y) \
                    .filter(lambda x: x[1] >= 50)

operation = lambda result, element: (min(result[0], element[0]), max(result[1], element[0]))

diffInPricesFrom2019 = salesFrom2019.join(filteredProducts).aggregateByKey((float('inf'), 0), operation, operation)\
                .mapValues(lambda x: x[1] - x[0]).cache()

diffInPricesFrom2019.reduce(lambda x,y: x if (x[1] > y[1]) else y)[0]

16

e) Idem anterior, pero calculando la categoría de
productos que registró mayor variación de precios en el último año.

In [213]:
diffInPricesFrom2019.join(infoRdd.map(lambda x: (x[0], x[2]))).map(lambda x: (x[1][1], abs(x[1][0]))) \
                    .reduceByKey(lambda x,y: x + y).reduce(lambda x,y: x if (x[1] > y[1]) else y)[0]

'Cosmetica'